In [2]:
import sys
sys.path.append('..')

from spyral.core.clusterize import form_clusters, join_clusters, cleanup_clusters
from spyral.core.run_stacks import form_run_string
from spyral import ClusterParameters

from e20009_phases.PointcloudLegacyPhase import PointCloud

from pathlib import Path
import h5py as h5
import numpy.random as random
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import DEFAULT_PLOTLY_COLORS

# Utility for syncing plot colors
def get_color(value: int) -> str:
    color_index = value
    if color_index >= len(DEFAULT_PLOTLY_COLORS):
        color_index = color_index % len(DEFAULT_PLOTLY_COLORS)
    elif color_index == -1:
        return "black"
    return DEFAULT_PLOTLY_COLORS[color_index]

NameError: name 'OverlapJoinParameters' is not defined

In [ ]:
# Load config
# workspace_path = Path("C:\\Users\\schaeffe\\Desktop\\B11_sim\\B9_2_79_a_Li5__106_MeV_beamm")


cluster_params = ClusterParameters(
    min_cloud_size=50,
    min_points=3,
    min_size_scale_factor=0.05,
    min_size_lower_cutoff=10,
    cluster_selection_epsilon=10.0,
    min_cluster_size_join=15,
    circle_overlap_ratio=0.25,
    outlier_scale_factor=0.05,
)

# pointcloud_path = workspace_path / "PointcloudLegacy"
pointcloud_path = Path("C:\\Users\\schaeffe\\Desktop\\e20009_B11_output\\PointcloudLegacy")

In [3]:
# Load data
run_number = 108
point_file_path = pointcloud_path / f"{form_run_string(run_number)}.h5"
point_file = h5.File(point_file_path, 'r')

cloud_group: h5.Group = point_file.get('cloud')
min_event = cloud_group.attrs['min_event']
max_event = cloud_group.attrs['max_event']

NameError: name 'pointcloud_path' is not defined

In [4]:
# Plot event
event = random.randint(min_event, max_event)
event = 590868
print(f'Event {event}')
event_data = cloud_group[f'cloud_{event}']
cloud = PointCloud()
cloud.load_cloud_from_hdf5_data(event_data[:].copy(), event)
print(f'Cloud size: {len(cloud.cloud)}')

fig = make_subplots(2,1,specs=[[{"type": "scene"}],[{"type": "xy"}]],row_heights=[0.6,0.4])
fig.add_trace(
    go.Scatter3d(
        x=cloud.cloud[:, 2], 
        y=cloud.cloud[:, 0], 
        z=cloud.cloud[:, 1], 
        mode="markers", 
        marker= {
            "size": 3, 
            "color": cloud.cloud[:, 3], 
            "showscale": True
        }, 
        name="Point Cloud"
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter(x=np.linalg.norm(cloud.cloud[:, :3], axis=1), y=cloud.cloud[:, 4], mode="markers", name="Charge"),
    row=2,
    col=1
)
fig.update_layout(
    xaxis_title="Distance (mm)",
    yaxis_title="Integrated Charge",
    scene = {
        "xaxis_range": [0.0, 1000.0],
        "yaxis_range": [-300.0, 300.0],
        "zaxis_range": [-300.0, 300.0],
        "xaxis_title": "Z (mm)",
        "yaxis_title": "X (mm)",
        "zaxis_title": "Y (mm)",
        "aspectratio": {
            "x": 3.3,
            "y": 1.0,
            "z": 1.0
        }
    },
    width=1300,
    height=1000,
)

Event 590868
Cloud size: 451


In [5]:
# Perform clustering
clusters, labels  = form_clusters(cloud, cluster_params)
total_points = 0
for cluster in clusters:
    total_points += len(cluster.point_cloud.cloud)
print(f"Size: {total_points}")

Size: 444


In [6]:
# Plot clusters
fig = make_subplots(2,1,specs=[[{"type": "scene"}],[{"type": "xy"}]],row_heights=[0.6,0.4])
scaled_data = np.zeros((len(cluster.point_cloud.cloud), 3))
for cluster in clusters:
    fig.add_trace(
        go.Scatter3d(
            x=cluster.point_cloud.cloud[:, 2], 
            y=cluster.point_cloud.cloud[:, 0], 
            z=cluster.point_cloud.cloud[:, 1], 
            mode="markers",
            legendgroup="clusters",
            marker= {
                "size": 3,
                "color": get_color(cluster.label)
            }, 
            name=f"Cluster {cluster.label}"
        ),
        row=1,
        col=1
    )
    fig.add_trace(
        go.Scatter(
            x=np.linalg.norm(cluster.point_cloud.cloud[:, :3], axis=1), 
            y=cluster.point_cloud.cloud[:, 3], 
            legendgroup="clusters",
            mode="markers",
            marker= {
                "color": get_color(cluster.label)
            },
            showlegend=False,
            name=f"Cluster {cluster.label}"
        ),
        row=2,
        col=1
    )
fig.update_layout(
    xaxis_title="Distance (mm)",
    yaxis_title="Integrated Charge",
    scene = {
        "xaxis_range": [0.0, 1000.0],
        "yaxis_range": [-300.0, 300.0],
        "zaxis_range": [-300.0, 300.0],
        "xaxis_title": "Z (mm)",
        "yaxis_title": "X (mm)",
        "zaxis_title": "Y (mm)",
        "aspectratio": {
            "x": 3.3,
            "y": 1.0,
            "z": 1.0
        }
    },
    width=1300,
    height=1000,
)

In [7]:
# Plot of scaled data the clustering algorithm actually saw when clustering
fig = go.Figure()
for cluster in clusters:
    fig.add_trace(
        go.Scatter3d(
            x=cluster.clustered_data[:, 2], 
            y=cluster.clustered_data[:, 0], 
            z=cluster.clustered_data[:, 1], 
            mode="markers",
            legendgroup="clusters",
            marker= {
                "size": 3,
                "color": get_color(cluster.label)
            }, 
            name=f"Cluster {cluster.label}"
        )
    )
fig.update_layout(
    scene = {
        "xaxis_title": "Z (arb)",
        "yaxis_title": "X (arb)",
        "zaxis_title": "Y (arb)",
        "aspectratio": {
            "x": 3.3,
            "y": 1.0,
            "z": 1.0
        }
    },
    width=1300,
    height=1000,
)

In [8]:
# Join broken cluster together
joined_clusters, labels = join_clusters(clusters, cluster_params, labels)

In [9]:
# Plot clusters after joining
fig = make_subplots(2,1,specs=[[{"type": "scene"}],[{"type": "xy"}]],row_heights=[0.6,0.4])
for cluster in joined_clusters:
    fig.add_trace(
        go.Scatter3d(
            x=cluster.point_cloud.cloud[:, 2], 
            y=cluster.point_cloud.cloud[:, 0], 
            z=cluster.point_cloud.cloud[:, 1], 
            mode="markers",
            legendgroup="clusters",
            marker= {
                "size": 3,
                "color": get_color(cluster.label)
            }, 
            name=f"Cluster {cluster.label}"
        ),
        row=1,
        col=1
    )
    fig.add_trace(
        go.Scatter(
            x=np.linalg.norm(cluster.point_cloud.cloud[:, :3], axis=1), 
            y=cluster.point_cloud.cloud[:, 4],
            legendgroup="clusters",
            mode="markers",
            marker= {
                "color": get_color(cluster.label)
            },
            showlegend=False,
            name=f"Cluster {cluster.label}"
        ),
        row=2,
        col=1
    )
fig.update_layout(
    xaxis_title="Distance (mm)",
    yaxis_title="Integrated Charge",
    scene = {
        "xaxis_range": [0.0, 1000.0],
        "yaxis_range": [-300.0, 300.0],
        "zaxis_range": [-300.0, 300.0],
        "xaxis_title": "Z (mm)",
        "yaxis_title": "X (mm)",
        "zaxis_title": "Y (mm)",
        "aspectratio": {
            "x": 3.3,
            "y": 1.0,
            "z": 1.0
        }
    },
    width=1300,
    height=1000,
)

In [10]:
# Remove outliers from clusters
cleaned_clusters, labels = cleanup_clusters(joined_clusters, cluster_params, labels)

In [11]:
# Plot cleaned up clusters
fig = make_subplots(2,1,specs=[[{"type": "scene"}],[{"type": "xy"}]],row_heights=[0.6,0.4])
for cluster in cleaned_clusters:
    fig.add_trace(
        go.Scatter3d(
            x=cluster.data[:, 2], 
            y=cluster.data[:, 0], 
            z=cluster.data[:, 1], 
            mode="markers",
            legendgroup="clusters",
            marker= {
                "size": 3,
                "color": get_color(cluster.label)
            }, 
            name=f"Cluster {cluster.label}"
        ),
        row=1,
        col=1
    )
    fig.add_trace(
        go.Scatter(
            x=np.linalg.norm(cluster.data[:, :3], axis=1), 
            y=cluster.data[:, 3],
            legendgroup="clusters",
            mode="markers",
            marker= {
                "color": get_color(cluster.label)
            },
            showlegend=False,
            name=f"Cluster {cluster.label}"
        ),
        row=2,
        col=1
    )
fig.update_layout(
    xaxis_title="Distance (mm)",
    yaxis_title="Integrated Charge",
    scene = {
        "xaxis_range": [0.0, 1000.0],
        "yaxis_range": [-300.0, 300.0],
        "zaxis_range": [-300.0, 300.0],
        "xaxis_title": "Z (mm)",
        "yaxis_title": "X (mm)",
        "zaxis_title": "Y (mm)",
        "aspectratio": {
            "x": 3.3,
            "y": 1.0,
            "z": 1.0
        }
    },
    width=1300,
    height=1000,
)